In [ ]:
%pip install tensorflow

In [ ]:
from IPython.display import display, clear_output
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd

import tensorflow as tf
print('Tensorflow version:', tf.__version__, '\n')

# Loading the MNIST dataset in one line
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize and reshape
x_train_normalized = x_train/255
x_test_normalized = x_test/255
x_train= x_train_normalized.reshape(-1, 28, 28, 1)
x_test = x_test_normalized.reshape(-1, 28, 28, 1)

# Printing the shape
print('x_train:', x_train.shape)
print('y_train:', y_train.shape)
print('x_test:', x_test.shape)
print('y_test:', y_test.shape)

In [ ]:
nets = 20
model = [0] * nets

for j in range(nets):

    model[j] = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(28,28,1)),

    tf.keras.layers.Conv2D(filters=28, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=28, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=28, kernel_size=5, strides=2, padding='same', activation='relu', use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.GaussianNoise(0.75),

    tf.keras.layers.Conv2D(filters=56, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=56, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=56, kernel_size=5, strides=2, padding='same', activation='relu', use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.GaussianNoise(0.75),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(82, kernel_regularizer = tf.keras.regularizers.l2(0.07), activation = 'relu'),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.GaussianNoise(0.75),

    tf.keras.layers.Dense(54, kernel_regularizer = tf.keras.regularizers.l2(0.07), activation = 'sigmoid'),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.GaussianNoise(0.75),

    tf.keras.layers.Dense(10, activation='softmax', use_bias=False)
    ])

    # Define your loss
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    # print(-tf.math.log(1/10))
    sampleID = 100
    loss_fn(y_train[:1], model[j](x_train[sampleID-1:sampleID]).numpy()).numpy()

    model[j].compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
                    loss= loss_fn,
                    metrics=['accuracy'])

vlr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.975 ** x)

er = tf.keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=10,
        restore_best_weights=True
)

In [ ]:
accuracies = []

for j in range(nets):
    
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=15,
                zoom_range = 0.15,
                shear_range=0.1,
                width_shift_range=0.1,
                height_shift_range=0.1,
                rescale=0,
                fill_mode = 'nearest',
                horizontal_flip=False,
                vertical_flip=False)
    datagen.fit(x_train)

    history = model[j].fit(
        datagen.flow(x_train, y_train, batch_size = 32),
        epochs=500,
        shuffle = True,
        validation_data=(x_test, y_test),
        callbacks=[vlr, er],
        verbose=1
    )

    (test_loss, test_acc) = model[j].evaluate(x_test, y_test)
    print("Loss: ", test_loss)
    print("Accuracy: ", test_acc)
    accuracies.append(test_acc)
    model_name = f'tmp/model.{j:d}.{test_acc:.4f}.h5'
    model[j].save(model_name, save_format='h5')

# Plotte die Genauigkeiten als Histogramm
plt.figure(figsize=(8, 6))
plt.hist(accuracies, bins=10, edgecolor='black')
plt.xlabel('Accuracy')
plt.ylabel('Frequency')
plt.title('Histogram of Model Accuracies')
plt.grid(True)
plt.show()

mean_accuracy = np.mean(accuracies)
std_accuracy = np.std(accuracies)

print(f'Mean Accuracy: {mean_accuracy:.4f}')
print(f'Standard Deviation of Accuracy: {std_accuracy:.4f}')

Weil das Training von 20 Model dieses großen Netzes s e h r lange dauert, sind nachfolgend Werte der Genauigkeit von 23 Durchläufen aufgelistet...

In [ ]:
accuracies = [0.9950000047683716,
              0.9973999857902527,
             0.9966999888420105,
             0.9962000250816345,
             0.9961000084877014,
             0.996999979019165,
             0.9965000152587891,
             0.9969000220298767,
             0.9959999918937683,
             0.9969000220298767,
             0.9962999820709229,
             0.9952999949455261,
             0.9965000152587891,
             0.9961000084877014,
             0.9958999752998352,
             0.9961000084877014,
             0.9966999888420105,
             0.996399998664856,
             0.9966999888420105,
             0.9952999949455261,
             0.9968000054359436,
             0.9965999722480774,
             0.9952999949455261]

# Plotte die Genauigkeiten als Histogramm
plt.figure(figsize=(8, 6))
plt.hist(accuracies, bins=10, edgecolor='black')
plt.xlabel('Accuracy')
plt.ylabel('Frequency')
plt.title('Histogram of Model Accuracies')
plt.grid(True)
plt.show()

mean_accuracy = np.mean(accuracies)
std_accuracy = np.std(accuracies)

print(f'Mean Accuracy: {mean_accuracy:.4f}')
print(f'Standard Deviation of Accuracy: {std_accuracy:.4f}')